In [1]:
import math

import numpy as np
import pandas as pd
import torch
import wandb as wandb

SEED = 3

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [2]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

In [3]:
from datetime import date

from torchvision import transforms

from util.dataset import FeaturePassengerFlowDataset
from util.transform import PandasToTensor, RollExogenousFeatures

transform = transforms.Compose([
    PandasToTensor(),
    RollExogenousFeatures()
])

train_data = FeaturePassengerFlowDataset(
    min_date=date(2022, 1, 1),
    max_date=date(2023, 1, 1),
    transform=transform)
validation_data = FeaturePassengerFlowDataset(
    min_date=date(2023, 1, 1),
    max_date=date(2023, 4, 1),
    transform=transform)
test_data = FeaturePassengerFlowDataset(
    min_date=date(2023, 4, 1),
    max_date=date(2023, 7, 1),
    transform=transform)

train_data._data

LOADING DATA: 100%|██████████| 14/14 [00:15<00:00,  1.13s/it]


passengers     noise  weekend  \
datetime            origin destination                                  
2022-01-01 00:00:00 12     19                159.0  1.788628     True   
                           LM                  6.0  0.436510     True   
                           OW                 25.0  0.096497     True   
                    16     24                 78.0 -1.863493     True   
                           CC                 82.0 -0.277388     True   
...                                            ...       ...      ...   
2022-12-31 23:00:00 WD     ED                 17.0 -1.369044     True   
                    WP     NC                 14.0 -1.853796     True   
                           PC                 35.0 -1.561378     True   
                    WS     FM                121.0 -0.871452     True   
                           ML                 16.0  1.432205     True   

                                        hours_00_04  hours_04_08  hours_08_12  \
datetime            origin destination                                          
2022-01-01 00:00:00 12     19                  True        False        False   
                           LM                  True        False        False   
                           OW                  True        False        False   
                    16     24                  True        False        False   
                           CC                  True        False        False   
...                                             ...          ...          ...   
2022-12-31 23:00:00 WD     ED                 False        False        False   
                    WP     NC                 False        False        False   
                           PC                 False        False        False   
                    WS     FM                 False        False        False   
                           ML                 False        False        False   

                                        hours_12_16  hours_16_20  hours_20_24  \
datetime            origin destination                                          
2022-01-01 00:00:00 12     19                 False        False        False   
                           LM                 False        False        False   
                           OW                 False        False        False   
                    16     24                 False        False        False   
                           CC                 False        False        False   
...                                             ...          ...          ...   
2022-12-31 23:00:00 WD     ED                 False        False         True   
                    WP     NC                 False        False         True   
                           PC                 False        False         True   
                    WS     FM                 False        False         True   
                           ML                 False        False         True   

                                        event_capacity  ...  \
datetime            origin destination                  ...   
2022-01-01 00:00:00 12     19                      0.0  ...   
                           LM                      0.0  ...   
                           OW                      0.0  ...   
                    16     24                      0.0  ...   
                           CC                      0.0  ...   
...                                                ...  ...   
2022-12-31 23:00:00 WD     ED                      0.0  ...   
                    WP     NC                      0.0  ...   
                           PC                      0.0  ...   
                    WS     FM                      0.0  ...   
                           ML                      0.0  ...   

                                        event_type_education  \
datetime            origin destination                         
2022-01-01 00:00:00 12     19     

In [4]:
from torch.utils.data import DataLoader

BATCH_SIZE = 256

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
validation_loader = DataLoader(validation_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [5]:
from torch.optim.lr_scheduler import ExponentialLR
from models.sarima import SARIMAX
from torch.optim import AdamW

import torch.nn as nn

device = torch.device('cuda:0')

model = SARIMAX(
    order=(23, 1, 0),
    seasonal_lag=24,
    seasonal_order=(7, 0, 0),
    n_features=train_data[0][0].shape[1] - 2
).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=1.0e-3, eps=1.0e-4)
scheduler = ExponentialLR(optimizer, 0.9)

In [6]:
import wandb

# Log run to Weights and Biases
wandb.init(project='passenger-flow-forecasting',
           config={
               'model': 'SARIMAX',
               'p': model._order[0],
               'd': model._order[1],
               'q': model._order[2],
               'P': model._seasonal_order[0],
               'D': model._seasonal_order[1],
               'Q': model._seasonal_order[2],
               'events': True,
               'features': train_data._data.columns
           })

wandb: Currently logged in as: jeffreybakker. Use `wandb login --relogin` to force relogin


In [7]:
from datetime import datetime
import os

from sklearn import metrics
from tqdm.notebook import tqdm

EPOCHS = 10
last_state = (-1, None)

# Create weights directories
os.makedirs('weights/sarimax/checkpoints', exist_ok=True)

tqdm_epoch = tqdm(desc='EPOCH', total=EPOCHS)
tqdm_batch = tqdm(desc='TRAIN', total=0)

for epoch in range(1, EPOCHS+1):
    # Set the model to training mode
    model.train()

    # Keep track of the training loss
    y_true = []
    y_pred = []

    # Loop over training data in batches
    tqdm_batch.reset(len(train_loader))
    tqdm_batch.desc = 'TRAIN'
    for batch in train_loader:
        # Move the data to the same device as the model
        history, horizon = tuple(t.to(device) for t in batch)

        # Select views of data
        y = horizon[:, 0, 0].squeeze()

        # Clear the gradients
        optimizer.zero_grad()

        # Compute outputs using a forward pass
        outputs = model(history, horizon).squeeze()

        # Compute the training loss of this batch
        loss = criterion(outputs, y)

        # Perform a backward pass to update weights
        loss.backward()
        optimizer.step()

        # Keep track of training loss
        y_true += y.cpu().numpy().tolist()
        y_pred += outputs.cpu().detach().numpy().tolist()

        tqdm_batch.update()

    train_mse = metrics.mean_squared_error(y_true, y_pred)
    train_mae = metrics.mean_absolute_error(y_true, y_pred)
    y_true = []
    y_pred = []

    # We don't need to keep track of gradients while testing on the validation set
    with torch.no_grad():
        model.eval()

        # Loop over data in batches
        tqdm_batch.reset(len(validation_loader))
        tqdm_batch.desc = 'VALIDATE'
        for batch in validation_loader:
            # Move the data to the same device as the model
            history, horizon = tuple(t.to(device) for t in batch)

            # Select views of data
            y = horizon[:, 0, 0].squeeze()

            # Compute outputs using a forward pass
            outputs = model(history, horizon).squeeze()

            # Keep track of validation loss
            y_true += y.cpu().numpy().tolist()
            y_pred += outputs.cpu().detach().numpy().tolist()

            tqdm_batch.update()

    validation_mse = metrics.mean_squared_error(y_true, y_pred)
    validation_mae = metrics.mean_absolute_error(y_true, y_pred)

    print(f'#{epoch:2d}    RMSE: {math.sqrt(validation_mse):.2f}    MAE: {validation_mae:.2f}')

    # Log metrics to Weights and Biases
    wandb.log({
        'train_mse': train_mse,
        'train_mae': train_mae,
        'validation_mse': validation_mse,
        'validation_mae': validation_mae
    }, commit=epoch < EPOCHS)

    scheduler.step()

    last_state = epoch, model.state_dict()

    if epoch < 5 or epoch % 5 == 0:
        torch.save(model.state_dict(), f'weights/sarimax/checkpoints/{epoch:2d}.pt')

    tqdm_epoch.update()

tqdm_batch.close()
tqdm_epoch.close()

EPOCH:   0%|          | 0/10 [00:00<?, ?it/s]

TRAIN: 0it [00:00, ?it/s]

# 1    RMSE: 140.90    MAE: 75.13
# 2    RMSE: 127.22    MAE: 60.89
# 3    RMSE: 127.54    MAE: 60.10
# 4    RMSE: 127.39    MAE: 59.56
# 5    RMSE: 127.78    MAE: 59.68
# 6    RMSE: 127.75    MAE: 59.07
# 7    RMSE: 127.77    MAE: 59.23
# 8    RMSE: 128.14    MAE: 59.26
# 9    RMSE: 128.61    MAE: 59.72
#10    RMSE: 128.29    MAE: 60.02


In [8]:
# Save the trained model
now = datetime.now()
datestring = f'{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}-{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}'
torch.save(last_state[1], f'weights/sarimax/{datestring}--{last_state[0]}.pt')
torch.save(last_state[1], f'weights/sarimax/seed-{SEED}.pt')
torch.save(last_state[1], f'weights/sarimax/latest.pt')

In [9]:
from os.path import exists

if exists(f'weights/sarimax/latest.pt'):
    model.load_state_dict(torch.load(f'weights/sarimax/latest.pt'))

In [10]:
import math

# Keep track of the loss
y_true = []
y_pred = []

# We don't need to keep track of gradients while testing
with torch.no_grad():
    model.eval()

    # Loop over data in batches
    for batch in tqdm(test_loader, desc='TEST'):
        # Move the data to the same device as the model
        history, horizon = tuple(t.to(device) for t in batch)

        # Select views of data
        y = horizon[:, 0, 0].squeeze()

        # Compute outputs using a forward pass
        outputs = model(history, horizon).squeeze()

        # Keep track of loss
        y_true += y.cpu().numpy().tolist()
        y_pred += outputs.cpu().detach().numpy().tolist()

        tqdm_batch.update()

# Cast results to integers
y_true = np.array(y_true).astype('int')
y_pred = np.array(y_pred).astype('int')

# Drop results where y_true == 0
mask = y_true > 0
y_true = y_true[mask]
y_pred = y_pred[mask]

test_mse = metrics.mean_squared_error(y_true, y_pred)
test_rmse = metrics.mean_squared_error(y_true, y_pred, squared=False)
test_mae = metrics.mean_absolute_error(y_true, y_pred)
test_mape = metrics.mean_absolute_percentage_error(y_true, y_pred)

# Log metrics to Weights and Biases
wandb.log({
    'mse': test_mse,
    'rmse': test_rmse,
    'mae': test_mae,
    'mape': test_mape
}, commit=True)

print(f'MSE: {test_mse:.2f}')
print(f'RMSE: {test_rmse:.2f}')
print(f'MAE: {test_mae:.2f}')
print(f'MAPE: {test_mape:.2f}')

TEST:   0%|          | 0/775 [00:00<?, ?it/s]

MSE: 20653.37
RMSE: 143.71
MAE: 67.84
MAPE: 1.13


In [11]:
# Finish the Weights and Biases run
wandb.finish()

mae,▁
mape,▁
mse,▁
rmse,▁
train_mae,█▂▁▁▁▁▁▁▁▁
train_mse,█▁▁▁▁▁▁▁▁▁
validation_mae,█▂▁▁▁▁▁▁▁▁
validation_mse,█▁▁▁▁▁▁▁▂▂
mae,67.83901
mape,1.12753
mse,20653.3707
